# Motion prediction applied to self-driving vehicles

## 1. Competition goal and introduction

Self-driving cars: where are we in 2020? A small video intro:

In [1]:
# a brief explanation is found in this video
from IPython.display import HTML


HTML('<center><iframe  width="850" height="450" src="https://www.youtube.com/watch?v=EzylsrXtkxI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')


The goal for this competition is then to predict the motion of extraneous vehicles, people, animals (called "agents").
Full details about the dateset can be found on this academic paper https://arxiv.org/pdf/2006.14480.pdf
In section 2 the most important concept will be introduced. 
In section 3 a predicting model will be proposed.

## 2. Import libraries and  explore data

Before importing/installing libraries I added the following datasets: kaggle-l5kit, lyft-config-files, lyft-resnet18-baseline

### 2.1 basic libraries and l5kit

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp

import gc
import os
from pathlib import Path
import random
import sys
import yaml
from tqdm.notebook import tqdm
import time




In [3]:
!jupyter nbconvert --version
!papermill --version


5.6.1
2.1.0 from /opt/conda/lib/python3.7/site-packages/papermill/cli.py (3.7.6)


In [4]:
#!pip install --no-index -f ../input/kaggle-l5kit pip==20.2.2 >/dev/nul
#!pip install --no-index -f ../input/kaggle-l5kit -U l5kit > /dev/nul

In [5]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0   9152      0 --:--:-- --:--:-- --:--:--  9135
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.5.0
Uninstalling torch-1.5.0:
  Successfully uninstalled torch-1.5.0
Found existing installation: torchvision 0.6.0a0+35d732a
Uninstalling torchvision-0.6.0a0+35d732a:
Done updating TPU runtime
  Successfully uninstalled torchvision-0.6.0a0+35d732a
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
| [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
\ [1 files][119.5 MiB/119.5 MiB]              

In [6]:
# TORCH XLA
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils

In [7]:
## this script transports l5kit and dependencies
os.system('pip uninstall typing -y')
os.system('pip install --ignore-installed --target=/kaggle/working l5kit')

# Hold back nbconvert to avoid https://github.com/jupyter/nbconvert/issues/1384
os.system('pip install --upgrade --ignore-installed --target=/kaggle/working "nbconvert==5.6.1"')

0

In [8]:

import l5kit
assert l5kit.__version__ == "1.1.0"

print ('l5kit imported')
print("l5kit version:", l5kit.__version__)

l5kit imported
l5kit version: 1.1.0


In [9]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16035 sha256=d857e1de0b1d12ee151fd4077f2f0bce46e909f6c175f17d985faec22f709ee2
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [10]:
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from itertools import islice
from typing import Dict


import torch
from torch import nn, optim


from efficientnet_pytorch import model as enet

from l5kit.evaluation.csv_utils import write_pred_csv
from l5kit.evaluation import compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory


from l5kit.configs import load_config_data


from tqdm import tqdm
from collections import Counter

from prettytable import PrettyTable


from IPython.display import display, clear_output
from IPython.display import HTML

import PIL
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')




In [11]:


# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold

from torch import nn
from typing import Dict
from pathlib import Path



from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### 2.2 Data import

The dataset consists of 170,000 scenes capturing the environment around the autonomous vehicle. Each scene encodes the state of the vehicle’s surroundings at a given point in time. 

In [12]:
l5kit_data_folder = "../input/lyft-motion-prediction-autonomous-vehicles"
os.environ["L5KIT_DATA_FOLDER"] = l5kit_data_folder


In [13]:
IMG_SIZE = 224
# --- Lyft configs ---
cfg = {
          'model_params': {'model_architecture': 'efficientnet-b0',
          'history_num_frames': 0,
          'history_step_size': 1,
          'history_delta_time': 0.1,
          'future_num_frames': 50,
          'future_step_size': 1,
          'future_delta_time': 0.1},

        'raster_params': {'raster_size': [IMG_SIZE, IMG_SIZE],
          'pixel_size': [0.5, 0.5],
          'ego_center': [0.25, 0.5],
          'map_type': 'py_semantic',
          'satellite_map_key': 'aerial_map/aerial_map.png',
          'semantic_map_key': 'semantic_map/semantic_map.pb',
          'dataset_meta_key': 'meta.json',
          'filter_agents_threshold': 0.5},

        'train_data_loader': {'key': 'scenes/train.zarr',
          'batch_size': 4,
          'shuffle': True,
          'num_workers': 0},

        "valid_data_loader":{"key": "scenes/validation.zarr",
                            "batch_size": 4,
                            "shuffle": False,
                            "num_workers": 0},
    
        "sample_data_loader": {
        'key': 'scenes/sample.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 0},
         
        "test_data_loader":{
        'key': "scenes/test.zarr",
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 0},

    
        'train_params': {'checkpoint_every_n_steps': 1000,
          'max_num_steps':3000,
          'eval_every_n_steps': 1000}
        }
print(cfg)

{'model_params': {'model_architecture': 'efficientnet-b0', 'history_num_frames': 0, 'history_step_size': 1, 'history_delta_time': 0.1, 'future_num_frames': 50, 'future_step_size': 1, 'future_delta_time': 0.1}, 'raster_params': {'raster_size': [224, 224], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5}, 'train_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 4, 'shuffle': True, 'num_workers': 0}, 'valid_data_loader': {'key': 'scenes/validation.zarr', 'batch_size': 4, 'shuffle': False, 'num_workers': 0}, 'sample_data_loader': {'key': 'scenes/sample.zarr', 'batch_size': 8, 'shuffle': False, 'num_workers': 0}, 'test_data_loader': {'key': 'scenes/test.zarr', 'batch_size': 8, 'shuffle': False, 'num_workers': 0}, 'train_params': {'checkpoint_every_n_steps': 1000, 'max_num_steps': 3000, 'ev

### 2.3 Data exploration

Credits to https://www.kaggle.com/nxrprime/lyft-understanding-the-data-baseline-on-tpus

Input data files are available in the read-only "../input/" directory.
The dataset is structured as follows:
- aerial_map
- scenes
- semantic_map

Aerial map subfolder contains images. Scenes subfolder contains information about foreing street objects classified in zarr files as following

- sample.zarr
- test.zarr
- train.zarr
- validate.zarr

For who never heard about "zarr" (for Italians, nothing to do with "zarro"):

"Zarr" provides classes and functions for working with N-dimensional arrays that behave like NumPy arrays but whose data is divided into chunks and each chunk is compressed.
Useful for handling big data.
More info at http://alimanfoo.github.io/2016/04/14/to-hdf5-and-beyond.html

The dataset consists of frames and agent states. A frame is a snapshot in time which consists of ego pose, time, and multiple agent states. Each agent state describes the position, orientation, bounds, and type.

sample.zarr data is used for visualization, train.zarr / validate.zarr / test.zarr are those needed for actual model training/validation/prediction.
We're building a LocalDataManager object. This will resolve relative paths from the config using the L5KIT_DATA_FOLDER env variable that was set above.

In [14]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["sample_data_loader"]["key"]) # for the EDA we use samples dataset. Smaller and RAM-ready
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)



+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |     316008    |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


#### Instead of working with raw data, L5Kit provides PyTorch ready datasets. It's much easier to use this wrapped dataset class to access data.

2 dataset class is implemented.

    EgoDataset: this dataset iterates over the AV (Autonomous Vehicle) annotations
    AgentDataset: this dataset iterates over other agents annotations

Let's see each class in detail. What kind of attributes/methods they have? What kind of data structure for each attributes?

As written in Class diagram, both classes are instantiated by:

    cfg: configuration file
    ChunkedDataset: Internal data class which holds 4 raw data scenes, frames, agents and tl_faces (described later).
    rasterizer: Rasterizer converts raw data into image.


### 2.4 Data examples

#### Objects in our maps, defined as "agents"

In [15]:
agents = zarr_dataset.agents
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=["label", "counts"])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

+------------------------------------+---------+
|               label                |  counts |
+------------------------------------+---------+
|      PERCEPTION_LABEL_NOT_SET      |    0    |
|      PERCEPTION_LABEL_UNKNOWN      | 1324481 |
|     PERCEPTION_LABEL_DONTCARE      |    0    |
|        PERCEPTION_LABEL_CAR        |  519385 |
|        PERCEPTION_LABEL_VAN        |    0    |
|       PERCEPTION_LABEL_TRAM        |    0    |
|        PERCEPTION_LABEL_BUS        |    0    |
|       PERCEPTION_LABEL_TRUCK       |    0    |
| PERCEPTION_LABEL_EMERGENCY_VEHICLE |    0    |
|   PERCEPTION_LABEL_OTHER_VEHICLE   |    0    |
|      PERCEPTION_LABEL_BICYCLE      |    0    |
|    PERCEPTION_LABEL_MOTORCYCLE     |    0    |
|      PERCEPTION_LABEL_CYCLIST      |   6688  |
|   PERCEPTION_LABEL_MOTORCYCLIST    |    0    |
|    PERCEPTION_LABEL_PEDESTRIAN     |  43182  |
|      PERCEPTION_LABEL_ANIMAL       |    0    |
|     AVRESEARCH_LABEL_DONTCARE      |    0    |
+-------------------

## 3. Predicting model

MODEL_NAME: It can be one of-

    "efficientnet-b0"
    "efficientnet-b1"
    "efficientnet-b2"
    "efficientnet-b3"
    "efficientnet-b4"
    "efficientnet-b5"
    "efficientnet-b6"
    "efficientnet-b7"

IMG_SIZE: I use 224 for efficientnet-b0



In [23]:
class Config:
    WEIGHT_FILE = "/kaggle/input/lyftpretrained-resnet101/lyft_efficientnetb0.pth" # Model state_dict path of previously trained model
    
    MODEL_NAME = "efficientnet-b0" # b0-b7 could be the different choices.
    
    IMG_SIZE = IMG_SIZE # stated above, together with cfg
    
    PIXEL_SIZE = 0.4
        
    EPOCHS = 2 # Epochs to train the model for.
    VALIDATION = True # A hyperparameter you could use to toggle for validating the model
    l_rate = 1e-4 # Learning rate

    scheduler_params = dict(
        mode='max',
        factor=0.7,
        patience=0,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-5,
        eps=1e-08
    )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau # Scheduler for learning rate.
    
    criterion = torch.nn.MSELoss(reduction="none") # Loss function.
     
    verbose_steps = 500 # Steps to print model's training status after.
    
config = Config()

In [24]:
def get_dataloader(config, zarr_data, map_type="py_semantic"):
    """Creates DataLoader instance for the given dataset."""
    cfg["raster_params"]["map_type"] = map_type
    rasterizer = build_rasterizer(cfg, dm)
    chunk_data = ChunkedDataset(zarr_data).open()
    agent_data = AgentDataset(cfg, chunk_data, rasterizer)
    dataloader = DataLoader(agent_data, 
                            batch_size=config["batch_size"],
                            num_workers=config["num_workers"],
                            shuffle=config["shuffle"]
                           )
    return dataloader

In [25]:
class TPUFitter:
    def __init__(self, model, device):
        self.model = model
        self.device = device

        # Following some lines are for setting up the AdamW optimizer.
        # See below explanation for details.
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
             'weight_decay': 0.001},
            
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0}
        ]

        self.optimizer = AdamW(optimizer_grouped_parameters, lr=config.l_rate*xm.xrt_world_size())
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        
        # Following function is used for printing to output efficiently. 
        xm.master_print(f'Model Loaded on {self.device}')

    def fit(self, train_loader, validation_loader):
        """Function to fit the model."""
        losses = []
        losses_mean = []
        
        progress = tqdm(range(cfg["train_params"]["max_num_steps"]))
        
        for e in range(config.EPOCHS):
            
            t = time.time() # Get a measurement of time.
            para_loader = pl.ParallelLoader(train_loader, [self.device]) # Distributed loading of the model.
            loss = self.forward(para_loader.per_device_loader(self.device))
            xm.master_print(
                            f'[RESULT]: Train. iter: {e+1}, ' + \
                            f'train_loss: {loss:.5f}, '+ \
                            f' time: {(time.time() - t)/60:.3f}'
                           )
            xm.master_print("\n")
            losses.append(loss.item())
            losses_mean.append(np.mean(losses))
           

    def validation(self, val_loader):
        """Function to validate the model's predictions."""
        val_losses = []
        val_losses_mean = []
        # Setting model to evaluation mode.
        self.model.eval()
        t = time.time()
        
        for step, data in enumerate(val_loader):
            with torch.no_grad():
                inputs = data["image"].to(self.device)
                targets = data["target_positions"].to(self.device)
                target_availabilities = data["target_availabilities"].unsqueeze(-1).to(self.device)
                
                outputs = self.model(inputs)
                outputs = outputs.reshape(targets.shape)
                val_loss = config.criterion(outputs, targets)
                val_loss = val_loss * target_availabilities
                val_loss = val_loss.mean()
                val_losses.append(val_loss.item())
                val_losses_mean.append(np.mean(val_losses))
                if step % config.verbose_steps == 0:
                    xm.master_print(
                        f'Valid Step {step}, val_loss: ' + \
                        f'{loss:.4f}' + \
                        f' time: {(time.time() - t)/60:.3f}'
                    )                
        return val_losses_mean
         
    def forward(self, train_loader):
        """Function to perform custom forward propagation."""
        
        # Setting model to training mode.
        self.model.train()
        
        t = time.time()
        for step, data in enumerate(train_loader):
            inputs = data["image"].to(self.device)
            target_availabilities = data["target_availabilities"].unsqueeze(-1).to(self.device)
            targets = data["target_positions"].to(self.device)
    
            outputs = self.model(inputs)
            outputs = outputs.reshape(targets.shape)
            loss = config.criterion(outputs, targets)
    
            loss = loss * target_availabilities
            loss = loss.mean()

            if step % config.verbose_steps == 0:
                xm.master_print(
                    f'Train Step {step}, loss: ' + \
                    f'{loss:.4f}' + \
                    f'time: {(time.time() - t)/60:.3f}'
                )
            self.optimizer.zero_grad()
        
            loss.backward()
            xm.optimizer_step(self.optimizer)
        
        self.model.eval()
        self.save('last-checkpoint.bin')
        return loss

    def save(self, path):
        """Function to save the model's current state."""
        xm.save(self.model.state_dict(), path)

In [26]:
# Implementation of class to load the particular EfficientNet model.
class LyftModel(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.backbone = enet.EfficientNet.from_name(config.MODEL_NAME)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        num_targets = 2*cfg["model_params"]["future_num_frames"]
    
        self.backbone._conv_stem = nn.Conv2d(
            num_in_channels,
            self.backbone._conv_stem.out_channels,
            kernel_size=self.backbone._conv_stem.kernel_size,
            stride=self.backbone._conv_stem.stride,
            padding=self.backbone._conv_stem.padding,
            bias=False
        )
    
        self.backbone._fc = nn.Linear(in_features=self.backbone._fc.in_features, out_features=num_targets)
    
    def forward(self, x):
        """Function to perform forward propagation."""
        x = self.backbone(x)
        return x

In [27]:
def get_dataloader(config, zarr_data, subset_len, map_type="py_semantic"):
    """Creates DataLoader instance for the given dataset."""
    
    cfg["raster_params"]["map_type"] = map_type
    rasterizer = build_rasterizer(cfg, dm)
    chunk_data = ChunkedDataset(zarr_data).open()
    agent_data = AgentDataset(cfg, chunk_data, rasterizer)
    
    # Getting Subset of the dataset.
    subset_data = torch.utils.data.Subset(agent_data, range(0, subset_len))
    
    dataloader = DataLoader(subset_data, 
                            batch_size=config["batch_size"],
                            num_workers=config["num_workers"],
                            shuffle=config["shuffle"]
                           )
    return dataloader

def train():
    device = xm.xla_device()
    model = LyftModel(cfg).to(device)
    fitter = TPUFitter(model, device)
    
    xm.master_print("Preparing the dataloader..")
    train_dataloader = get_dataloader(cfg["train_data_loader"], dm.require("scenes/train.zarr"), 40000)
    val_dataloader   = get_dataloader(cfg["valid_data_loader"], dm.require("scenes/validate.zarr"), 3000)
    
    xm.master_print("Training the model..")
    fitter.fit(train_dataloader, val_dataloader)
    fitter.save("lyft_model.pth")
    return fitter

In [28]:

model = train()


Model Loaded on xla:1
Preparing the dataloader..



  0%|          | 0/3000 [00:00<?, ?it/s]

Training the model..
Train Step 0, loss: 86.2797time: 0.192
Train Step 500, loss: 34.0370time: 2.757
Train Step 1000, loss: 9.3204time: 4.793
Train Step 1500, loss: 7.7972time: 6.815
Train Step 2000, loss: 21.1679time: 8.811
Train Step 2500, loss: 2.0109time: 10.809
Train Step 3000, loss: 45.8703time: 12.822
Train Step 3500, loss: 1.0292time: 14.801
Train Step 4000, loss: 0.7298time: 16.803
Train Step 4500, loss: 0.5520time: 18.793
Train Step 5000, loss: 13.6099time: 20.797
Train Step 5500, loss: 5.6880time: 22.778
Train Step 6000, loss: 23.2177time: 24.780
Train Step 6500, loss: 10.3101time: 26.778
Train Step 7000, loss: 3.6012time: 28.762
Train Step 7500, loss: 2.6846time: 30.745
Train Step 8000, loss: 12.9188time: 32.743
Train Step 8500, loss: 14.5902time: 34.740
Train Step 9000, loss: 2.6042time: 36.721
Train Step 9500, loss: 4.1083time: 38.706
[RESULT]: Train. iter: 1, train_loss: 2.79391,  time: 40.667


Train Step 0, loss: 14.0732time: 0.013
Train Step 500, loss: 7.0381time: 2.0

In [29]:
#print("Saving the model...")
#torch.save(model.state_dict(), "lyft_model.pth")

In [30]:
PATH_TO_DATA = '/kaggle/input/lyft-motion-prediction-autonomous-vehicles/'

test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{PATH_TO_DATA}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataset)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning:

you're running with a custom agents_mask



+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   11314    |  1131400   |  88594921  |    7854144    |      31.43      |        100.00        |        78.31         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [31]:
def _test():
    

    future_coords_offsets_pd = []
    timestamps = []
    agent_ids = []
    device = 'xla:0'
    print(f"device: {device} ready!")
    model = LyftModel(cfg)
    ckpt = torch.load('../input/lyftmodelall/effnet0l2binay_368.bin')
    model.load_state_dict(ckpt)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        dataiter = tqdm(test_dataloader)

        for data in dataiter:
            inputs = data["image"].to(device)
            target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
            targets = data["target_positions"].to(device)

            outputs = model(inputs).reshape(targets.shape)

            future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
            timestamps.append(data["timestamp"].numpy().copy())
            agent_ids.append(data["track_id"].numpy().copy())
    write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))

In [32]:
def forward(data, model, device):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    return outputs

def _test():
    future_coords_offsets_pd = []
    timestamps = []
    agent_ids = []
    device = 'xla:0'
    print(f"device: {device} ready!")
    model = LyftModel(cfg)
    ckpt = torch.load('./lyft_model.pth')
    model.load_state_dict(ckpt)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        
    # store information for evaluation
        future_coords_offsets_pd = []
        timestamps = []
        agent_ids = []

        progress_bar = tqdm(test_dataloader)
        for data in progress_bar:

        # convert agent coordinates into world offsets
            agents_coords = forward(data, model, device).cpu().numpy().copy()
            world_from_agents = data["world_from_agent"].numpy()
            centroids = data["centroid"].numpy()
            coords_offset = []

        for agent_coords, world_from_agent, centroid in zip(agents_coords, world_from_agents, centroids):
            coords_offset.append(transform_points(agent_coords, world_from_agent) - centroid[:2])
            future_coords_offsets_pd.append(np.stack(coords_offset))
            timestamps.append(data["timestamp"].numpy().copy())
            agent_ids.append(data["track_id"].numpy().copy())


    write_pred_csv(
        "submission.csv",
        timestamps=np.concatenate(timestamps),
        track_ids=np.concatenate(agent_ids),
        coords=np.concatenate(future_coords_offsets_pd),
    )


In [ ]:
def _mp_fn(rank, flags):
#     torch.set_default_tensor_type('torch.FloatTensor')
    _test()

FLAGS={}

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=1, start_method='fork')

device: xla:0 ready!




  0%|          | 0/8891 [00:00<?, ?it/s]

  0%|          | 1/8891 [00:18<46:41:23, 18.91s/it]

  0%|          | 2/8891 [00:19<33:14:19, 13.46s/it]

  0%|          | 3/8891 [00:20<23:42:10,  9.60s/it]

  0%|          | 4/8891 [00:20<17:01:18,  6.90s/it]

  0%|          | 5/8891 [00:21<12:21:22,  5.01s/it]

  0%|          | 6/8891 [00:22<9:05:02,  3.68s/it] 

  0%|          | 7/8891 [00:22<6:47:47,  2.75s/it]

  0%|          | 8/8891 [00:23<5:08:08,  2.08s/it]

  0%|          | 9/8891 [00:23<4:01:57,  1.63s/it]

  0%|          | 10/8891 [00:24<3:20:11,  1.35s/it]

  0%|          | 11/8891 [00:25<2:50:58,  1.16s/it]

  0%|          | 12/8891 [00:25<2:32:51,  1.03s/it]

  0%|          | 13/8891 [00:26<2:19:41,  1.06it/s]

  0%|          | 14/8891 [00:27<2:09:14,  1.14it/s]

  0%|          | 15/8891 [00:27<1:59:32,  1.24it/s]

  0%|          | 16/8891 [00:28<1:55:09,  1.28it/s]

  0%|          | 17/8891 [00:29<1:49:08,  1.36it/s]

  0%|          | 18/8891 [00:29<1:42:17,  1.45it/s]

  0%|

  2%|▏         | 138/8891 [01:43<1:31:29,  1.59it/s]

  2%|▏         | 139/8891 [01:43<1:30:55,  1.60it/s]

  2%|▏         | 140/8891 [01:44<1:29:15,  1.63it/s]

  2%|▏         | 141/8891 [01:45<1:25:28,  1.71it/s]

  2%|▏         | 142/8891 [01:45<1:28:42,  1.64it/s]

  2%|▏         | 143/8891 [01:46<1:30:39,  1.61it/s]

  2%|▏         | 144/8891 [01:46<1:29:15,  1.63it/s]

  2%|▏         | 145/8891 [01:47<1:28:26,  1.65it/s]

  2%|▏         | 146/8891 [01:48<1:27:32,  1.66it/s]

  2%|▏         | 147/8891 [01:48<1:26:51,  1.68it/s]

  2%|▏         | 148/8891 [01:49<1:26:35,  1.68it/s]

  2%|▏         | 149/8891 [01:49<1:25:58,  1.69it/s]

  2%|▏         | 150/8891 [01:50<1:25:38,  1.70it/s]

  2%|▏         | 151/8891 [01:51<1:25:00,  1.71it/s]

  2%|▏         | 152/8891 [01:51<1:24:42,  1.72it/s]

  2%|▏         | 153/8891 [01:52<1:25:17,  1.71it/s]

  2%|▏         | 154/8891 [01:52<1:23:07,  1.75it/s]

  2%|▏         | 155/8891 [01:53<1:27:40,  1.66it/s]

  2%|▏         | 156/8891 [0

  3%|▎         | 274/8891 [03:07<1:31:10,  1.58it/s]

  3%|▎         | 275/8891 [03:08<1:29:19,  1.61it/s]

  3%|▎         | 276/8891 [03:08<1:27:56,  1.63it/s]

  3%|▎         | 277/8891 [03:09<1:26:43,  1.66it/s]

  3%|▎         | 278/8891 [03:09<1:26:12,  1.67it/s]

  3%|▎         | 279/8891 [03:10<1:25:43,  1.67it/s]

  3%|▎         | 280/8891 [03:11<1:25:26,  1.68it/s]

  3%|▎         | 281/8891 [03:11<1:25:39,  1.68it/s]

  3%|▎         | 282/8891 [03:12<1:27:05,  1.65it/s]

  3%|▎         | 283/8891 [03:12<1:25:48,  1.67it/s]

  3%|▎         | 284/8891 [03:13<1:25:54,  1.67it/s]

  3%|▎         | 285/8891 [03:14<1:25:41,  1.67it/s]

  3%|▎         | 286/8891 [03:14<1:25:24,  1.68it/s]

  3%|▎         | 287/8891 [03:15<1:25:19,  1.68it/s]

  3%|▎         | 288/8891 [03:15<1:24:50,  1.69it/s]

  3%|▎         | 289/8891 [03:16<1:21:25,  1.76it/s]

  3%|▎         | 290/8891 [03:17<1:23:51,  1.71it/s]

  3%|▎         | 291/8891 [03:17<1:28:10,  1.63it/s]

  3%|▎         | 292/8891 [0

  5%|▍         | 410/8891 [04:31<1:28:53,  1.59it/s]

  5%|▍         | 411/8891 [04:32<1:28:25,  1.60it/s]

  5%|▍         | 412/8891 [04:32<1:27:16,  1.62it/s]

  5%|▍         | 413/8891 [04:33<1:24:57,  1.66it/s]

  5%|▍         | 414/8891 [04:33<1:23:12,  1.70it/s]

  5%|▍         | 415/8891 [04:34<1:24:33,  1.67it/s]

  5%|▍         | 416/8891 [04:35<1:25:24,  1.65it/s]

  5%|▍         | 417/8891 [04:35<1:25:46,  1.65it/s]

  5%|▍         | 418/8891 [04:36<1:22:54,  1.70it/s]

  5%|▍         | 419/8891 [04:36<1:23:30,  1.69it/s]

  5%|▍         | 420/8891 [04:37<1:23:18,  1.69it/s]

  5%|▍         | 421/8891 [04:38<1:23:08,  1.70it/s]

  5%|▍         | 422/8891 [04:38<1:24:31,  1.67it/s]

  5%|▍         | 423/8891 [04:39<1:23:55,  1.68it/s]

  5%|▍         | 424/8891 [04:39<1:23:38,  1.69it/s]

  5%|▍         | 425/8891 [04:40<1:22:19,  1.71it/s]

  5%|▍         | 426/8891 [04:41<1:22:34,  1.71it/s]

  5%|▍         | 427/8891 [04:41<1:26:04,  1.64it/s]

  5%|▍         | 428/8891 [0

...

 73%|███████▎  | 6521/8891 [1:08:34<28:15,  1.40it/s]

 73%|███████▎  | 6522/8891 [1:08:34<28:07,  1.40it/s]

 73%|███████▎  | 6523/8891 [1:08:35<27:28,  1.44it/s]

 73%|███████▎  | 6524/8891 [1:08:36<26:47,  1.47it/s]

 73%|███████▎  | 6525/8891 [1:08:36<25:33,  1.54it/s]

 73%|███████▎  | 6526/8891 [1:08:37<24:56,  1.58it/s]

 73%|███████▎  | 6527/8891 [1:08:37<25:12,  1.56it/s]

 73%|███████▎  | 6528/8891 [1:08:38<25:25,  1.55it/s]

 73%|███████▎  | 6529/8891 [1:08:39<25:36,  1.54it/s]

 73%|███████▎  | 6530/8891 [1:08:39<25:01,  1.57it/s]

 73%|███████▎  | 6531/8891 [1:08:40<24:22,  1.61it/s]

 73%|███████▎  | 6532/8891 [1:08:40<24:11,  1.63it/s]

 73%|███████▎  | 6533/8891 [1:08:41<24:22,  1.61it/s]

 73%|███████▎  | 6534/8891 [1:08:42<24:34,  1.60it/s]

 74%|███████▎  | 6535/8891 [1:08:42<24:56,  1.57it/s]

 74%|███████▎  | 6536/8891 [1:08:43<24:40,  1.59it/s]

 74%|███████▎  | 6537/8891 [1:08:44<23:44,  1.65it/s]

 74%|███████▎  | 6538/8891 [1:08:44<22:43,  1.73it/s]

 74%|█████

 75%|███████▍  | 6655/8891 [1:09:58<23:55,  1.56it/s]

 75%|███████▍  | 6656/8891 [1:09:59<23:21,  1.59it/s]

 75%|███████▍  | 6657/8891 [1:09:59<23:26,  1.59it/s]

 75%|███████▍  | 6658/8891 [1:10:00<22:50,  1.63it/s]

 75%|███████▍  | 6659/8891 [1:10:00<22:58,  1.62it/s]

 75%|███████▍  | 6660/8891 [1:10:01<23:44,  1.57it/s]

 75%|███████▍  | 6661/8891 [1:10:02<23:56,  1.55it/s]

 75%|███████▍  | 6662/8891 [1:10:03<24:09,  1.54it/s]

 75%|███████▍  | 6663/8891 [1:10:03<24:36,  1.51it/s]

 75%|███████▍  | 6664/8891 [1:10:04<25:23,  1.46it/s]

 75%|███████▍  | 6665/8891 [1:10:05<25:45,  1.44it/s]

 75%|███████▍  | 6666/8891 [1:10:05<25:50,  1.44it/s]

 75%|███████▍  | 6667/8891 [1:10:06<25:46,  1.44it/s]

 75%|███████▍  | 6668/8891 [1:10:07<25:16,  1.47it/s]

 75%|███████▌  | 6669/8891 [1:10:07<24:23,  1.52it/s]

 75%|███████▌  | 6670/8891 [1:10:08<23:01,  1.61it/s]

 75%|███████▌  | 6671/8891 [1:10:08<21:54,  1.69it/s]

 75%|███████▌  | 6672/8891 [1:10:09<21:49,  1.69it/s]

 75%|█████

 76%|███████▋  | 6789/8891 [1:11:22<23:44,  1.48it/s]

 76%|███████▋  | 6790/8891 [1:11:23<23:35,  1.48it/s]

 76%|███████▋  | 6791/8891 [1:11:23<22:43,  1.54it/s]

 76%|███████▋  | 6792/8891 [1:11:24<21:29,  1.63it/s]

 76%|███████▋  | 6793/8891 [1:11:25<20:43,  1.69it/s]

 76%|███████▋  | 6794/8891 [1:11:25<21:04,  1.66it/s]

 76%|███████▋  | 6795/8891 [1:11:26<21:11,  1.65it/s]

 76%|███████▋  | 6796/8891 [1:11:26<21:15,  1.64it/s]

 76%|███████▋  | 6797/8891 [1:11:27<21:41,  1.61it/s]

 76%|███████▋  | 6798/8891 [1:11:28<21:32,  1.62it/s]

 76%|███████▋  | 6799/8891 [1:11:28<21:09,  1.65it/s]

 76%|███████▋  | 6800/8891 [1:11:29<20:52,  1.67it/s]

 76%|███████▋  | 6801/8891 [1:11:29<20:54,  1.67it/s]

 77%|███████▋  | 6802/8891 [1:11:30<21:01,  1.66it/s]

 77%|███████▋  | 6803/8891 [1:11:31<20:53,  1.67it/s]

 77%|███████▋  | 6804/8891 [1:11:31<21:03,  1.65it/s]

 77%|███████▋  | 6805/8891 [1:11:32<21:24,  1.62it/s]

 77%|███████▋  | 6806/8891 [1:11:32<21:22,  1.63it/s]

 77%|█████

 78%|███████▊  | 6923/8891 [1:12:46<19:47,  1.66it/s]

 78%|███████▊  | 6924/8891 [1:12:46<19:27,  1.68it/s]

 78%|███████▊  | 6925/8891 [1:12:47<19:14,  1.70it/s]

 78%|███████▊  | 6926/8891 [1:12:47<19:25,  1.69it/s]

 78%|███████▊  | 6927/8891 [1:12:48<19:38,  1.67it/s]

 78%|███████▊  | 6928/8891 [1:12:49<19:23,  1.69it/s]

 78%|███████▊  | 6929/8891 [1:12:49<18:36,  1.76it/s]

 78%|███████▊  | 6930/8891 [1:12:50<17:47,  1.84it/s]

 78%|███████▊  | 6931/8891 [1:12:50<18:41,  1.75it/s]

 78%|███████▊  | 6932/8891 [1:12:51<19:09,  1.70it/s]

 78%|███████▊  | 6933/8891 [1:12:52<20:34,  1.59it/s]

 78%|███████▊  | 6934/8891 [1:12:52<21:18,  1.53it/s]

 78%|███████▊  | 6935/8891 [1:12:53<20:47,  1.57it/s]

 78%|███████▊  | 6936/8891 [1:12:54<20:23,  1.60it/s]

 78%|███████▊  | 6937/8891 [1:12:54<20:05,  1.62it/s]

 78%|███████▊  | 6938/8891 [1:12:55<19:52,  1.64it/s]

 78%|███████▊  | 6939/8891 [1:12:55<19:40,  1.65it/s]

 78%|███████▊  | 6940/8891 [1:12:56<19:36,  1.66it/s]

 78%|█████

 79%|███████▉  | 7057/8891 [1:14:10<19:17,  1.58it/s]

 79%|███████▉  | 7058/8891 [1:14:11<19:26,  1.57it/s]

 79%|███████▉  | 7059/8891 [1:14:11<19:28,  1.57it/s]

 79%|███████▉  | 7060/8891 [1:14:12<18:58,  1.61it/s]

 79%|███████▉  | 7061/8891 [1:14:12<17:51,  1.71it/s]

 79%|███████▉  | 7062/8891 [1:14:13<17:36,  1.73it/s]

 79%|███████▉  | 7063/8891 [1:14:14<18:22,  1.66it/s]

 79%|███████▉  | 7064/8891 [1:14:14<19:15,  1.58it/s]

 79%|███████▉  | 7065/8891 [1:14:15<20:04,  1.52it/s]

 79%|███████▉  | 7066/8891 [1:14:16<19:32,  1.56it/s]

 79%|███████▉  | 7067/8891 [1:14:16<18:54,  1.61it/s]

 79%|███████▉  | 7068/8891 [1:14:17<18:44,  1.62it/s]

 80%|███████▉  | 7069/8891 [1:14:17<18:35,  1.63it/s]

 80%|███████▉  | 7070/8891 [1:14:18<18:32,  1.64it/s]

 80%|███████▉  | 7071/8891 [1:14:19<18:28,  1.64it/s]

 80%|███████▉  | 7072/8891 [1:14:19<18:36,  1.63it/s]

 80%|███████▉  | 7073/8891 [1:14:20<18:29,  1.64it/s]

 80%|███████▉  | 7074/8891 [1:14:20<18:17,  1.66it/s]

 80%|█████

The trained efficientnet is finally run on the test dataset and the submission.csv file has been created, ready to be submitted